In [1]:
import requests
import json
import pprint
import pandas as pd
import csv
import re
from datetime import datetime

- Get all shops

In [2]:
all_shops_API = requests.get("https://www.foody.vn/__get/Place/HomeListPlace?page=6&lat=10.823099&lon=106.629664&count=500&districtId=&cateId=&cuisineId=&type=1", 
                            headers={'X-Requested-With': 'XMLHttpRequest',
                                            'Cookie': 'floc=217'})
data = all_shops_API.text
dataJson = json.loads(data)
print(len(dataJson['Items']))

0


In [7]:
def extractshop(page, count, lat, long):
    while count > 0:
        url = f'https://www.foody.vn/__get/Place/HomeListPlace?page={page}&lat={lat}&lon={long}&count={count}&districtId=&cateId=&cuisineId=&type=1'
        all_shops_API = requests.get(url, headers={'X-Requested-With': 'XMLHttpRequest',
                                                'Cookie': 'floc=217'})
        data = all_shops_API.text
        dataJson = json.loads(data)
        if len(dataJson['Items']) != 0:
            break
        else: 
            count = count - 1    
           
    
    shop_review = []
    CityId = dataJson['CityId']
    
    for shop in dataJson['Items']: 
        converted_date = shop['ResCreatedOn']
        converted_date = converted_date[7:-2]
        converted_date = int(converted_date)
        converted_date = datetime.fromtimestamp(converted_date // 1000000000)
        converted_date = converted_date.strftime('%d/%m/%Y')
        shop_review.append({'CityId': CityId, 
                            'name': shop['Name'],
                            'addr': shop['Address'],
                            'RestaurantId': shop['RestaurantId'],
                            'RestaurantStatus': shop['RestaurantStatus'],
                            'avgrating' : shop['AvgRating'],
                            'TotalPictures' : shop['TotalPictures'],
                            'TotalSaves' : shop['TotalSaves'],
                            'TotalReviews' : shop['TotalReviews'],
                            'TotalFavourites' : shop['TotalFavourites'],
                            'TotalViews' : shop['TotalViews'],
                            'TotalCheckIns' : shop['TotalCheckIns'],
                            'CreatedOn'  : converted_date,
                            'Latitude'  : shop['Latitude'],
                            'Longitude'  : shop['Longitude'],
                            'ResUrlReviews' : shop['ResUrlReviews']})
    return shop_review

def extractshop_all(lat, long):
    n = 1
    page = 0
    all_shop = []
    while n == 1:
        one_page = extractshop(page, 500, lat, long)
        all_shop.extend(one_page)
        if len(one_page) < 500:
            break
        else:
            page += 1
    return all_shop

In [9]:
lmao = extractshop_all(10.823099, 106.629664)
lmao

2900

In [10]:
df = pd.DataFrame.from_dict(lmao) 
df.to_csv (r'allshop.csv', index = False, header=True, encoding='utf-8')

- Get menu of ONE shop: fải loop qua từng request_id trong cell code trên

In [ ]:
# VD: có thể thay request_id = 1031704 để thử
request_id = 1031704
delivery_dish_API = requests.get(f"https://gappapi.deliverynow.vn/api/dish/get_delivery_dishes?request_id={request_id}&id_type=1", 
                                    headers={'x-foody-api-version': '1',
                                            'x-foody-app-type': '1004',
                                            'x-foody-client-language': 'vi',
                                            'x-foody-client-type': '1',
                                            'x-foody-client-version': '1',
                                            'X-Foody-Client-Id': 'cookies.__ondemand_sessionid'
                                        }
                                )
data = delivery_dish_API.text
dataJson = json.loads(data)
print(dataJson)

- get detail of ONE shop

In [ ]:
delivery_dish_API = requests.get("https://gappapi.deliverynow.vn/api/delivery/get_detail?request_id={request_id}&id_type=1", 
                                    headers={'x-foody-api-version': '1',
                                            'x-foody-app-type': '1004',
                                            'x-foody-client-language': 'vi',
                                            'x-foody-client-type': '1',
                                            'x-foody-client-version': '1',
                                            'X-Foody-Client-Id': 'cookies.__ondemand_sessionid'
                                        }
                                )
data = delivery_dish_API.text
dataJson = json.loads(data)
print(dataJson)